# Theseus

Theseus is a library for differentiable nonlinear optimization built on PyTorch. 

Theseus is motivated by problems in robotics and computer vision that can be formulated as differentiable nonlinear least squares optimization problems, such as Simultaneous Localization and Mapping (SLAM), motion planning, and bundle adjustment. These problems can be broadly categorized as doing <i>structured learning</i>, where neural components can be modularly mixed with known priors to get the benefit of deep learning in a way that adds value over classical methods. While interest in this area is rapidly increasing, existing work is fragmented and built using application-specific codebases. Theseus fills this gap by providing a problem-agnostic platform for structured learning, letting the user easily combine neural networks with priors represented as differentiable blocks of nonlinear optimization problems, and do end-to-end training over these. 

This tutorial introduces the basic building blocks for solving such optimization problems in Theseus; in the following tutorials, we will show how to put together these building blocks to solve optimization problems with various aspects and increasing complexity. We cover six conceptual building blocks in this tutorial: 
* **Variables:** named wrappers for torch tensors that form the fundamental data type for defining optimization problems in Theseus. (Section 1)
* **Cost functions:** computes an error term as a function of one or more variables, and which are the functions to be minimized by Theseus optimizers. (Section 2)
* **Cost weights:** computes a weight that modifies the contribution of one or more cost functions to the overall objective. (Section 3)
* **Objective:** compiles multiple cost functions and weights to define the structure of an optimization problem. (Section 4)
* **Optimizer:** implements an optimization algorithm (e.g., Gauss-Newton, LevenbergMarquardt) that can be used to minimize an Objective. (Section 5)
* **TheseusLayer:** groups an objective and optimizer and serves as the interface between torch modules upstream/downstream and differentiable optimization problems. (Section 6)

## 1. Variables

Optimization objectives in Theseus are a function of `th.Variable` objects, which are `torch.tensor` wrappers of different types (e.g., 2D points, rotation groups, etc.) that can be, optionally, associated with a name. In Theseus, we <i>require</i> the first dimension of all variables to be a batch dimension (similar to the convention in PyTorch modules). We describe here two main operations common to all `Variables`: (1) Creating variables and (2) Updating `Variables`.

### 1.1  Creating variables
Variables can be created using the generic `th.Variable` interface, or through a sub-class with custom functionality. Many `Variables` used in Theseus applications are manifolds; therefore, Theseus provides several `Variable` sub-classes supporting commonly used manifolds, such as vectors, 2-D/3-D points, 2-D rotations and 2-D rigid transformations. We show some example usage below:

In [1]:
import torch
import theseus as th

In [2]:
# Create a variable with 3-D random data of batch size = 2 and name "x"
x = th.Variable(torch.randn(2, 3), name="x")
print(f"x: Named variable with 3-D data of batch size 2:\n  {x}\n")

# Create an unnamed variable. A default name will be created for it
y = th.Variable(torch.zeros(1, 1))
print(f"y: Un-named variable:\n  {y}\n")

# Create a named SE2 (2D rigid transformation) specifying data (batch_size=2)
z = th.SE2(x_y_theta=torch.zeros(2, 3).double(), name="se2_1")
print(f"z: Named SE2 variable:\n  {z}")

x: Named variable with 3-D data of batch size 2:
  Variable(tensor=tensor([[-0.5966,  0.7318,  2.2279],
        [ 0.6040,  0.3843, -2.0580]]), name=x)

y: Un-named variable:
  Variable(tensor=tensor([[0.]]), name=Variable__1)

z: Named SE2 variable:
  SE2(xytheta=tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64), name=se2_1)


### 1.2  Updating variables

After creating a variable, its value can be updated via `update()` method. Below we show a few examples and possible errors to avoid when updating variables.

In [3]:
# Example usage of `update`
print("Example usage of `update`: ")
print(f"  Original variable: {x}")
x.update(torch.ones(2, 3))
print(f"  Updated variable: {x}\n")

# The following inputs don't work
print("Error inputs for a Variable `update`:")
try:
    # `update` expects input tensor to respect the internal data format
    x.update(torch.zeros(2, 4))
except ValueError as e:
    print(f"  Mismatched internal data format:")
    print(f"    {e}")
try:
    # `update` expects a batch dimension
    x.update(torch.zeros(3))
except ValueError as e:
    print(f"  Missing batch dimension: ")
    print(f"    {e}\n")
    
# However the batch size can be changed via `update`
print("Change variable batch size via `update`:")
x.update(torch.ones(4, 3))
print(f"  New shape: {x.shape}")

Example usage of `update`: 
  Original variable: Variable(tensor=tensor([[-0.5966,  0.7318,  2.2279],
        [ 0.6040,  0.3843, -2.0580]]), name=x)
  Updated variable: Variable(tensor=tensor([[1., 1., 1.],
        [1., 1., 1.]]), name=x)

Error inputs for a Variable `update`:
  Mismatched internal data format:
    Tried to update tensor x with data incompatible with original tensor shape. Given torch.Size([4]). Expected: torch.Size([3])
  Missing batch dimension: 
    Tried to update tensor x with data incompatible with original tensor shape. Given torch.Size([]). Expected: torch.Size([3])

Change variable batch size via `update`:
  New shape: torch.Size([4, 3])


Over the next few sections, we will see the different ways that `Variable`s are used in optimization problems in Theseus.

## 2. Cost functions

A Theseus cost function represents an error function of one or more Theseus variables. Thus, cost functions capture the core quantities being optimized in Theseus.

For this reason, a cost function needs to know which variables can be optimized, and which variables are not allowed to be optimized. In Theseus, we represent this concept by having two kinds of variables: 
* *optimization variables*: variables that can be modified by Theseus optimizers for minimizing the objective. 
* *auxiliary variables*: variables that are required to compute the objective, but that remain constant to Theseus optimizers.
 
In Theseus, a `Variable` becomes an optimization variable if it is defined as such in the creation of a cost function. All optimization variables must be sub-classes of `th.Manifold`.

A cost function thus needs to be created with its optimization (required) and auxiliary variables (optional) declared. The core operations provided by a cost function are the computation of the error and the error's Jacobian using the latest values of its variables. The `th.CostFunction` class is an abstract class, and to instantiate it, one needs to implement the error computation and the Jacobian. A cost function must return a `torch` tensor as its error. 

As a simple example, we will show how to use the `th.Difference` cost function, which is a concrete sub-class of `th.CostFunction`. Below, we instantiate this cost function with two `Vector` variables, one optimization and one auxiliary. 

We then show a few useful operations on the cost function: how the cost function can access its optimization and auxiliary variables; the computation of its error, which is defined as `optim_var - target` for the `th.Difference` c); how the error changes when an underlying `Variable` is updated. Lastly, we show the computation of its jacobians: this returns a list of jacobians, with one entry per *optimization* variable.

In [4]:
# Note: CostWeight is a weighting quantity required for constructing a cost function.
# We explain it in Section 3; for this example, we simply create it but we do not use it.
w1 = th.ScaleCostWeight(2.0)

# Create a Difference cost function
optim_var = th.Vector(tensor=torch.ones(1, 2), name="x1")
target = th.Vector(tensor=torch.zeros(1, 2), name="target")
cf = th.Difference(optim_var, target, w1)

# A cost function can retrieve its optimization and auxiliary variables 
print ("Retrieving the optimization and auxiliary variables from the cost function:")
print("  Optimization variables: ", list(cf.optim_vars))
print("  Auxiliary variables: ", list(cf.aux_vars))
print("")

# Cost functions compute the error using the values of the variables.
error = cf.error()
print(f"Original cost function (unweighted) error:\n  {error} of shape {error.shape}\n")

# Cost functions use the _latest_ values of the variables,
# as shown by the error values after the variable is updated.
print("Updating optimization variables by factor of 2: ")
optim_var.update(2 * torch.ones(1, 2))
print(f"  Updated variables: {optim_var}")
# Error is now twice as large as the one printed above
print(f"  Updated (unweighted) error: {cf.error()}\n")

# Compute the (unweighted) jacobians and error
# This returns a list of jacobians, with one entry per _optimization_ variable.
print("Computing cost function's (unweighted) jacobians:")
jacobians, error = cf.jacobians()  # Note cf.jacobians also returns error 
print(f"  Jacobians: {type(jacobians)} of length {len(jacobians)}")
print(f"    {jacobians[0]}")
# The i-th jacobian has shape (batch_size, cf.dim(), i-th_optim_var.dof())
print(f"    Shape of 0-th Jacobian: {jacobians[0].shape}")

Retrieving the optimization and auxiliary variables from the cost function:
  Optimization variables:  [Vector(dof=2, tensor=tensor([[1., 1.]]), name=x1)]
  Auxiliary variables:  [Vector(dof=2, tensor=tensor([[0., 0.]]), name=target)]

Original cost function (unweighted) error:
  tensor([[1., 1.]]) of shape torch.Size([1, 2])

Updating optimization variables by factor of 2: 
  Updated variables: Vector(dof=2, tensor=tensor([[2., 2.]]), name=x1)
  Updated (unweighted) error: tensor([[2., 2.]])

Computing cost function's (unweighted) jacobians:
  Jacobians: <class 'list'> of length 1
    tensor([[[1., 0.],
         [0., 1.]]])
    Shape of 0-th Jacobian: torch.Size([1, 2, 2])


In Tutorial 3, we will delve into the internals of a cost function and show how to construct custom cost functions.

## 3. Cost weights

The Theseus *cost weight* is a weighting function applied to cost functions: it computes a weight as a function of one or more variables, and applies it to the error of one or more cost functions. The cost weights are thus a way of modifying the error of a cost function in the optimization problem. Cost weights add another layer of abstraction that help trade-off between different cost functions in an objective.

The `th.CostWeight` class is abstract, as any function of `Variable`s can be used to create `CostWeight`. Theseus provides a number of concrete `CostWeight` sub-classes currently:  
- `ScaleCostWeight`, where the weighting function is a scalar real number, 
- `DiagonalCostWeight`, where the the weighting function is a diagonal matrix,
- `th.eb.GPCostWeight`, where the weighting function represents the inverse covariance function of an [exactly sparse Gaussian process](http://roboticsproceedings.org/rss10/p01.pdf). 

The main use of the `CostWeight` is to support the `weighted_error` and `weighted_jacobians_and_error` functions of the cost functions; so these sub-classes implement their (defined) weighting functions.

The `Variable`s used in a `CostWeight` may be named or unnamed; however, using a named `Variable` allows us to update the value of the `CostWeight` directly; this is especially useful in updating the `Objective` or the `TheseusLayer` whenever the cost weight is computed by some external function (e.g., a `torch.nn.Module`). 

We show an example of `CostWeight` usage below with the `ScaleCostWeight` class. 

In [5]:
print("Scale cost weight creation:")
# Create a scale cost weight from a float
w1 = th.ScaleCostWeight(10.0)
# The weight is wrapped into a default variable
print(f"  w1 (default variable): {w1.scale}")

# A theseus variable can be passed directly
w2 = th.ScaleCostWeight(th.Variable(2 * torch.ones(1, 1), name="scale"))
print(f"  w2 (named variable): {w2.scale}\n")

# Weighting errors and jacobians with a ScaleCostWeight
print("Weighting errors/jacobian directly with a ScaleCostWeight:")
weighted_jacobians, weighted_error = w1.weight_jacobians_and_error(jacobians, error)
print(f"  jacobians:\n     weighted: {weighted_jacobians}\n     original: {jacobians}")
print(f"  error:\n    weighted: {weighted_error}\n    original: {error}\n")

# If the ScaleCostWeight is included in the cost function, we can directly
# use the `weight_errors` and `weight_jacobians_and_error` of the cost function.
print("Using the `weighted_error` function of the previous cost function:") 
print(f"  weighted cost function error: {cf.weighted_error()} vs unweighted error: {cf.error()}")

Scale cost weight creation:
  w1 (default variable): Variable(tensor=tensor([[10.]]), name=Variable__17)
  w2 (named variable): Variable(tensor=tensor([[2.]]), name=scale)

Weighting errors/jacobian directly with a ScaleCostWeight:
  jacobians:
     weighted: [tensor([[[10.,  0.],
         [ 0., 10.]]])]
     original: [tensor([[[1., 0.],
         [0., 1.]]])]
  error:
    weighted: tensor([[20., 20.]])
    original: tensor([[2., 2.]])

Using the `weighted_error` function of the previous cost function:
  weighted cost function error: tensor([[4., 4.]]) vs unweighted error: tensor([[2., 2.]])


## 4. Objective

A `th.Objective` defines the structure of an optimization problem, by adding one or more cost functions to it, each with associated cost weights and variables. The `th.Objective` will combine them into a global error function, with an internal structure that can be used by a Theseus optimizer to minimize the global error via changes in the optimization variables. 

Currently, `th.Objective` supports nonlinear sum of squares objectives, where the global error is the sum of the squares of each of its cost function errors, weighted by their corresponding cost weights. We plan to extend to other optimization structures in the future. A critical point in the creation of the objective is that **Theseus assumes that cost weights provided will also be squared in the final the objective.** Formally, we currently support objectives of the form

<p align="center">
    <img src="https://raw.githubusercontent.com/facebookresearch/theseus/main/tutorials/fig/theseus_objective.png?token=ABEKAIXVTXL7BBFIRBLAEHTBWJ2R2" alt="Theseus Objective" width="250"/>
</p>

where **v** represents the set of variables, *f*<sub>i</sub> is a cost function error, and *w*<sub>i</sub> its associated cost weight.

Below we show a simple example of creating an objective. We will want to minimize the following function <i>(x - a)<sup>2</sup> + 4(y - b)<sup>2</sup></i>, where *a* and *b* as constants, *x* and *y* as variables. Below, we first create (1) the optimization and auxiliary variables, (2) cost weights, (3) cost functions, (4) objective.

Then, to evaluate the `Objective`, we will use its `error_squared_norm` function. Before we can evaluate it, however, we must use the `Objective.update` function at least once (so that the internal data structures are correctly set up). In general, the `update` function is used to easily change the values of all variables registered with the `Objective`. This function receives a dictionary that maps variable names to torch tensors to which the corresponding variables should be updated. 

We finally show that the current objective is computed correctly for this function. (In the next section, we optimize the objective to its minimum value)

In [6]:
# Step 1: Construct optimization and auxiliary variables.
# Construct variables of the function: these the optimization variables of the cost functions. 
x = th.Vector(1, name="x")
y = th.Vector(1, name="y")

# Construct auxiliary variables for the constants of the function.
a = th.Vector(tensor=torch.randn(1,1), name="a")
b = th.Vector(tensor=torch.randn(1,1), name="b")

# Step 2: Construct cost weights
# For w1, let's use a named variable
w1 = th.ScaleCostWeight(th.Variable(tensor=torch.ones(1, 1), name="w1_sqrt"))
w2 = th.ScaleCostWeight(2.0)  # we provide 2, as sqrt of 4 for the (y-b)^2 term

# Step 3: Construct cost functions representing each error term
# First term
cf1 = th.Difference(x, a, w1, name="term_1")
# Second term
cf2 = th.Difference(y, b, w2, name="term_2")

# Step 4: Create the objective function and add the error terms
objective = th.Objective()
objective.add(cf1)
objective.add(cf2)

# Step 5: Evaluate objective under current values
# Note this needs to be preceded by a call to `objective.update`
# Here we use the update function to set values of all variables
objective.update({"a": torch.ones(1,1), "b": 2 * torch.ones(1, 1), 
                  "x": 0.5 * torch.ones(1,1), "y": 3 * torch.ones(1, 1)})
# Weighted error should be: cost_weight * weighted_error 
print(f"Error term 1: unweighted: {cf1.error()} weighted: {cf1.weighted_error()}")
print(f"Error term 2: unweighted: {cf2.error()} weighted: {cf2.weighted_error()}")
# Objective value should be: (error1)^2 + (error2)^2 
print(f"Objective value: {objective.error_squared_norm()}")

Error term 1: unweighted: tensor([[-0.5000]]) weighted: tensor([[-0.5000]])
Error term 2: unweighted: tensor([[1.]]) weighted: tensor([[2.]])
Objective value: tensor([4.2500])


Adding cost functions to the objective registers all of its optimization and auxiliary variables (and those of its cost weights, if present). `th.Objective` also checks that names are not overloaded by different variable or cost function objects

In [7]:
try:
    objective.add(th.Difference(y, b, w2, name="term_1"))
except ValueError as e:
    print(e)
    
try:
    obj2 = th.Objective()
    obj2.add(th.Difference(x, a, w1, name="term_1"))
    fake_x1 = th.Vector(1, name="x")
    obj2.add(th.Difference(fake_x1, b, w2, name="fake_term"))
except ValueError as e:
    print(e)

Two different cost function objects with the same name (term_1) are not allowed in the same objective.
Two different variable objects with the same name (x) are not allowed in the same objective.


## 5. Optimizers

Theseus provides a set of linear and nonlinear optimizers for minimizing problems described as `th.Objective`. 
The objective can be solved by calling `optimizer.optimize()`, which will change the values of optimization 
variables to minimize its associated objective. `optimize` leaves the optimization variables at the final values found, 
and returns an info object about the optimization (which contains the best solution and optimization statistics).

In [8]:
# Recall that our objective is (x - a)^2 + 4 (y - b)^2
# which is minimized at x = a and y = b
# Let's start by assigning random values to them
objective.update({
    "x": torch.randn(1, 1),
    "y": torch.randn(1, 1)
})

# Now let's use the optimizer. Because this problem is minimizing a
# quadratic form, a linear optimizer can solve for the optimal solution
optimizer = th.LinearOptimizer(objective, th.CholeskyDenseSolver)
info = optimizer.optimize()

# Now let's check the values of x and y 
# Here we print only the Vectors' tensor attributes for ease of understanding
print(f"x: {x.tensor} vs a: {a.tensor}")  # Matches a = 1
print(f"y: {y.tensor} vs b: {b.tensor}")  # Matches b = 2
print(f"Objective after optimization: {objective.error_squared_norm()}")

x: tensor([[1.]]) vs a: tensor([[1.]])
y: tensor([[2.]]) vs b: tensor([[2.]])
Objective after optimization: tensor([0.])


/private/home/lep/code/theseus/theseus/optimizer/optimizer.py:42: UserWarning: Vectorization is off by default when not running from TheseusLayer. Using TheseusLayer is the recommended way to run our optimizers.
  warnings.warn(


## 6. TheseusLayer

As the warning above indicates, the recommended way to run our optimizers is via `TheseusLayer`. The `TheseusLayer` provides an interface between `torch` code upstream/downstream, and Theseus objectives and optimizers. The `forward()` method combines the functionality of `Objective.update()` and `Optimizer.optimizer()` into a single call. It receives an update dictionary as input, and returns a dictionary with the torch data of optimization variables after optimization, as well as the optimizer's output info.

In [9]:
layer = th.TheseusLayer(optimizer)
values, info = layer.forward({
    "x": torch.randn(1, 1),
    "y": torch.randn(1, 1),
    "a": torch.ones(1, 1),
    "b": 2 * torch.ones(1, 1),
    "w1_sqrt": torch.ones(1, 1)
})
print(f"After calling TheseusLayer's forward():")
print(f"  Values: {values}")
print(f"  Info: {info}")
print(f"  Optimized objective: {objective.error_squared_norm()}")

After calling TheseusLayer's forward():
  Values: {'x': tensor([[1.]]), 'y': tensor([[2.]])}
  Info: OptimizerInfo(best_solution={'x': tensor([[1.]]), 'y': tensor([[2.]])}, status=array([<LinearOptimizerStatus.CONVERGED: 1>], dtype=object))
  Optimized objective: tensor([0.])


The `TheseusLayer` allows for backpropagation, and is semantically similar to a layer in a PyTorch neural network. Backpropagating through the `TheseusLayer` allows for learning of any necessary quantities of the problem, such as cost weights, initial values for the optimization variables, and other parameters for the optimization. The following tutorials will illustrate several applications for learning with a `TheseusLayer`.

To distinguish between the optimization done by the Theseus optimizers, and those done outside the Theseus optimizers (e.g., by PyTorch's autograd during learning), we will refer to them as *inner loop optimization* and *outer loop optimization*, respectively. Note that the inner loop optimization optimizes only the optimization variables, and the outer loop optimization can optimize torch tensors associated with selected variables provided to the PyTorch autograd optimizers. A call to `TheseusLayer` `forward()` performs only inner loop optimization; typically the PyTorch autograd learning steps will perform the outer loop optimizations. We will see examples of this in the following tutorials.

During the outer loop, we will commonly want to update Theseus variables before running inner loop optimization; for example, to set initial values for optimization variables, or to update auxiliary variables with tensors learned by the outer loop. We recommend that such updates to Theseus variables are done via `TheseusLayer.forward()`. While variables and objectives can be updated independently without going through `TheseusLayer.forward()`, following this convention makes it explicitly what the latest inputs to the `TheseusLayer` are, helping to avoid hidden errors and unwanted behavior. Therefore, we recommend that any updates during learning be performed only via the `TheseusLayer`.